In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
np

<module 'numpy' from '/opt/conda/lib/python3.7/site-packages/numpy/__init__.py'>

In [6]:
# Reading input

articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [7]:
pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', nrows = 3)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [8]:
pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv', nrows = 3)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...


In [9]:
pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', nrows = 3)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2


In [10]:
# articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
# customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [11]:
transactions_train['max_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('max')

transactions_train['min_purc_date'] = transactions_train.groupby('customer_id')['t_dat'].transform('min')

transactions_train['total_visits'] = transactions_train.groupby('customer_id')['t_dat'].transform('nunique')

transactions_train['total_item_bought'] = transactions_train.groupby('customer_id')['article_id'].transform('nunique')

transactions_train['max_year'] = transactions_train['max_purc_date'].str.slice(0,4).astype(int)

In [12]:
transactions_train.shape

(31788324, 10)

In [13]:
np.percentile(transactions_train[['customer_id','total_item_bought']].drop_duplicates()['total_item_bought'], 25)

3.0

In [14]:
transactions_train[['customer_id','total_item_bought']].drop_duplicates()['total_item_bought'].describe().transpose()

count    1.362281e+06
mean     2.004464e+01
std      3.198025e+01
min      1.000000e+00
25%      3.000000e+00
50%      8.000000e+00
75%      2.400000e+01
max      1.346000e+03
Name: total_item_bought, dtype: float64

In [15]:
transactions_train[['customer_id','total_item_bought']].drop_duplicates()['total_item_bought'].mean()

20.04464497412795

In [16]:
transactions_train_filt = transactions_train[transactions_train['max_year'] == 2020]

transactions_train_filt = transactions_train_filt[transactions_train_filt['total_visits'] >= 10]

In [17]:
transactions_train_filt.shape

(20464700, 10)

In [18]:
transactions_train_filt = transactions_train_filt[transactions_train_filt['total_item_bought'] >= 10]

In [19]:
transactions_train_filt.shape

(20462951, 10)

In [20]:
transactions_train_filt['product_reach'] = transactions_train_filt.groupby('article_id')['customer_id'].transform('nunique')

In [21]:
np.percentile(transactions_train_filt[['article_id','product_reach']].drop_duplicates()['product_reach'], 25)

8.0

In [22]:
transactions_train_filt[['article_id','product_reach']].drop_duplicates()['product_reach'].describe()

count    101577.000000
mean        171.534324
std         395.757739
min           1.000000
25%           8.000000
50%          38.000000
75%         172.000000
max       17949.000000
Name: product_reach, dtype: float64

In [23]:
transactions_train_filt = transactions_train_filt[transactions_train_filt['product_reach'] > 8]

In [24]:
transactions_train_filt.shape

(20358515, 11)

In [27]:
transactions_train_filt.head()

,t_dat,customer_id,article_id,price,sales_channel_id,max_purc_date,min_purc_date,total_visits,total_item_bought,max_year,product_reach
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,2020-05-13,2018-09-20,16,110,2020,33
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,2020-05-13,2018-09-20,16,110,2020,1144
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,2020-05-13,2018-09-20,16,110,2020,1133
5,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,0.016932,2,2020-05-13,2018-09-20,16,110,2020,923
6,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,0.020322,2,2020-05-13,2018-09-20,16,110,2020,49


In [ ]:
transactions_train_filt

In [28]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [37]:
customers.groupby('postal_code').agg({'customer_id':'nunique'}).reset_index().sort_values('customer_id', ascending=False)

,postal_code,customer_id
61034,2c29ae653a9282cce4151bd87643c907644e09541abc28...,120303
281937,cc4ed85e30f4977dae47662ddc468cd2eec11472de6fac...,261
156090,714976379549eb90aae4a71bca6c7402cc646ae7c40f6c...,159
171208,7c1fa3b0ec1d37ce2c3f34f63bd792f3b4494f324b6be5...,157
126228,5b7eb31eabebd3277de632b82267286d847fd5d44287ee...,156
...,...,...
129744,5e14f1b9db2fc96d22ed0a48462e684061ee81ab82b36e...,1
271330,c4873af1061bd5be842d810d9d8549941befa4f4aaf56a...,1
129746,5e153c2cb31cce8575346a7b165527a58d49d5223cfd64...,1
271328,c48723f620fb40095368752f0dc577b5575dea418bea6e...,1


In [38]:
Post_code_cust = customers.groupby('postal_code').agg({'customer_id':'nunique'}).reset_index()

In [40]:
Post_code_cust.groupby('customer_id').agg({'postal_code':'count'}).reset_index().sort_values('postal_code', ascending = False)

,customer_id,postal_code
0,1,90368
1,2,74900
2,3,57229
3,4,41364
4,5,28572
...,...,...
84,91,1
80,87,1
79,85,1
65,66,1


In [34]:
customers['FN'].sort_values().unique()

array([ 1., nan])

In [35]:
customers['Active'].sort_values().unique()

array([ 1., nan])

In [31]:
customers['age'].sort_values().unique()

array([16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
       55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67.,
       68., 69., 70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80.,
       81., 82., 83., 84., 85., 86., 87., 88., 89., 90., 91., 92., 93.,
       94., 95., 96., 97., 98., 99., nan])

In [32]:
customers['club_member_status'].sort_values().unique()

array(['ACTIVE', 'LEFT CLUB', 'PRE-CREATE', nan], dtype=object)

In [33]:
customers['fashion_news_frequency'].sort_values().unique()

array(['Monthly', 'NONE', 'None', 'Regularly', nan], dtype=object)

In [ ]:
selected_customers = pd.merge(transactions_train,customer_group, 
                              left_on = 'customer_id',
                              right_on = 'customer_id',
                              how = 'inner'
                              
                             )

In [ ]:
import gc
del transactions_train
gc.collect()

In [ ]:
selected_customers['prod_count'] = selected_customers.groupby('customer_id')['article_id'].transform('nunique')

In [ ]:
selected_customers.head()

In [ ]:
selected_customers[['prod_count']].describe().transpose()

In [ ]:
selected_customers['article_id'].nunique()

In [ ]:
np.percentile(selected_customers[['prod_count']], 95)

In [ ]:
selected_customers = selected_customers[selected_customers['prod_count'].between(2,172)]

In [ ]:
selected_customers['customer_id'].nunique()

In [ ]:
selected_customers.head()

In [43]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [64]:
sample_submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [ ]:
pd.options.display()

In [70]:
pd.set_option('display.max_colwidth', -1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [71]:
sample_submission['prediction'].iloc[:3]

0    0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001
1    0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001
2    0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001
Name: prediction, dtype: object

In [46]:
sample_submission['customer_id'].nunique()

1371980

In [47]:
transactions_train.shape

(31788324, 10)

In [61]:
top_products = transactions_train[transactions_train['max_year'] == 2020].groupby(['article_id']).agg({"customer_id":"count"}).reset_index()

In [72]:
top_12_prodcuts = top_products.sort_values('customer_id', ascending = False).iloc[12:24]

In [73]:
top_12_prodcuts['article_id'].tolist()

[562245046,
 448509014,
 751471001,
 673396002,
 351484002,
 673677002,
 399256001,
 562245001,
 160442007,
 568601006,
 688537004,
 608776002]

In [ ]:
706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001

In [75]:
# articles[articles['article_id'].isin(top_12_prodcuts['article_id'].tolist())]

In [77]:
repurchsed_top_items = transactions_train.groupby(['customer_id','article_id']).agg({'t_dat':'nunique'}).reset_index()

In [96]:
prod_list = repurchsed_top_items[repurchsed_top_items['t_dat'] > 16].groupby('article_id').agg({"customer_id":"count"}).\
reset_index().sort_values('customer_id', ascending=False).iloc[:12]['article_id'].unique()

In [103]:
products = ''
for c, i in enumerate(prod_list):
    if c < 11:
        products = products+"0"+str(i)+' '
    else:
        products = products+"0"+str(i)
        

In [104]:
products

'0120129001 0189616001 0228257001 0228257002 0228257003 0399223001 0581298001 0668537001 0668537002 0706016001 0728162001 0758043001'